## Set Up

In [6]:
import psycopg2
import pandas as pd
import numpy as np
import seaborn as sns
from  scipy.stats import ttest_ind
import matplotlib.pyplot as plt
%matplotlib inline
conn = psycopg2.connect("dbname=musicdatabase")
cur = conn.cursor()

In [9]:
#GENRES:
cur.execute("SELECT * FROM genres")
names = [description[0] for description in cur.description]
genres = cur.fetchall()
genres_df = pd.DataFrame(genres, columns=names)

# #ARTISTS
cur.execute("SELECT * FROM artists")
names = [description[0] for description in cur.description]
artists = cur.fetchall()
artists_df = pd.DataFrame(artists, columns=names)

# #CONTENT
cur.execute("SELECT * FROM content")
names = [description[0] for description in cur.description]
content = cur.fetchall()
content_df = pd.DataFrame(content, columns=names)

# #LABELS
cur.execute("SELECT * FROM labels")
names = [description[0] for description in cur.description]
labels = cur.fetchall()
labels_df = pd.DataFrame(labels, columns=names)

# #REVIEWS
cur.execute("SELECT * FROM reviews")
names = [description[0] for description in cur.description]
reviews = cur.fetchall()
reviews_df = pd.DataFrame(reviews, columns=names)

# #YEARS
cur.execute("SELECT * FROM years")
names = [description[0] for description in cur.description]
years = cur.fetchall()
years_df = pd.DataFrame(years, columns=names)

## Clean Up

One hot encoding genres table:

In [10]:
genres_one_hot = pd.get_dummies(genres_df['genre'])
genres_one_hot.head()
genres_hot_df = pd.concat([genres_df, genres_one_hot], axis = 1)
genres_hot_df.head()

,reviewid,genre,electronic,experimental,folk/country,global,jazz,metal,pop/r&b,rap,rock
0,22703,electronic,1,0,0,0,0,0,0,0,0
1,22721,metal,0,0,0,0,0,1,0,0,0
2,22659,rock,0,0,0,0,0,0,0,0,1
3,22661,rock,0,0,0,0,0,0,0,0,1
4,22725,electronic,1,0,0,0,0,0,0,0,0


Filtering genres table for ONLY pop and electronic genres:

In [11]:
genres_hot_df_filtered = genres_hot_df[(genres_hot_df['pop/r&b'] == 1) | (genres_hot_df['electronic'] == 1)]
genres_hot_df_filtered = genres_hot_df_filtered[['reviewid', 'electronic', 'pop/r&b']]
genres_hot_df_filtered.head()

,reviewid,electronic,pop/r&b
0,22703,1,0
4,22725,1,0
7,22694,1,0
12,22700,1,0
18,22719,0,1


Adding ```value_counts()``` to genres table -- counts # of genres per album:

In [12]:
genre_counts = genres_hot_df_filtered['reviewid'].value_counts()
genre_dict = genre_counts.to_dict()
genres_hot_df_filtered['value_count'] = genres_hot_df_filtered['reviewid'].apply(lambda x: genre_dict[x])
genres_hot_df_filtered.head()

,reviewid,electronic,pop/r&b,value_count
0,22703,1,0,1
4,22725,1,0,1
7,22694,1,0,1
12,22700,1,0,1
18,22719,0,1,1


Drop all albums with > 1 genre assigned to it:

In [13]:
genre_to_drop = genres_hot_df_filtered[genres_hot_df_filtered['value_count'] == 2]
genres_df_new = genres_hot_df_filtered.drop(genre_to_drop.index, axis=0)
genres_df_new.head()

,reviewid,electronic,pop/r&b,value_count
0,22703,1,0,1
4,22725,1,0,1
7,22694,1,0,1
12,22700,1,0,1
18,22719,0,1,1


Merge the new genre table with the existing review table:

In [14]:
#rg = review & genres
rg_df = reviews_df.merge(genres_df_new, left_on='reviewid', right_on='reviewid')
rg_df.head()

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,electronic,pop/r&b,value_count
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017,1,0,1
1,22725,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017,1,0,1
2,22694,tehillim,yotam avni,http://pitchfork.com/reviews/albums/22694-tehi...,7.0,0,andy beta,contributor,2017-01-05,3,5,1,2017,1,0,1
3,22700,nadir,steven warwick,http://pitchfork.com/reviews/albums/22700-nadir/,7.6,0,thea ballard,contributor,2017-01-03,1,3,1,2017,1,0,1
4,22719,merry christmas lil mama,"chance the rapper, jeremih",http://pitchfork.com/reviews/albums/22719-merr...,8.1,0,sheldon pearce,associate staff writer,2016-12-30,4,30,12,2016,0,1,1


## Functions

In [17]:
def t_and_p_stats (df1, df2):
    print('df1 Review Mean: ', df1['score'].mean())
    print('df2 Review Mean: ', df2['score'].mean())
    print('Difference: ', df1['score'].mean() - df2['score'].mean())
    t_stat, p_val = ttest_ind(df1['score'],df2['score'])
    print(f"T-Statistic: {t_stat}\nP-Value: {p_val}")

In [18]:
def sample_size(df1, df2, moe=.5):
    std_df1 = df1['score'].std()
    std_df2 = df2['score'].std()
    count_df1 = len(df1['score'])
    count_df2 = len(df2['score'])
    
    std_p = (( (count_df1-1)*(std_df1**2) ) + ( (count_df2-1)* (std_df2**2) )) / (count_df1 + count_df2 - 2)
    samp_size = 2*( ((1.96 * std_p)/moe)**2 )
    return samp_size

In [19]:
def bootstrap(df1, df2, moe=0.5):
    bootstrap_df1_sample = []
    bootstrap_df2_sample = []

    bootstrap_df1_sample_means = np.zeros(100)
    bootstrap_df2_sample_means = np.zeros(100)

    for i in range(100):
        bootstrap_df1_samp = np.random.choice(df1['score'], size=int(sample_size(df1, df2, moe)), replace=True)
        bootstrap_df2_samp = np.random.choice(df2['score'], size=int(sample_size(df1, df2, moe)), replace=True)
    
        bootstrap_df1_sample.append(bootstrap_df1_samp)
        bootstrap_df2_sample.append(bootstrap_df2_samp)
    
        bootstrap_df1_sample_means[i] = bootstrap_df1_samp.mean()
        bootstrap_df2_sample_means[i] = bootstrap_df2_samp.mean()

    print('df1 Means Stdv: ', bootstrap_df1_sample_means.std())
    print('df2 Means Stdv: ', bootstrap_df2_sample_means.std())

    print('df1 2.5 Percentile: ', np.percentile(a=bootstrap_df1_sample_means, q=2.5))
    print('df1 97.5 Percentile: ', np.percentile(a=bootstrap_df1_sample_means, q=97.5))

    print('df2 2.5 Percentile: ', np.percentile(a=bootstrap_df2_sample_means, q=2.5))
    print('df2 97.5 Percentile: ', np.percentile(a=bootstrap_df2_sample_means, q=97.5))

## Dataframes for Analysis

In [ ]:
pop_df = rg_df[rg_df['pop/r&b'] == 1].copy()
elec_df = rg_df[rg_df['electronic'] == 1].copy()

In [20]:
t_and_p_stats(pop_df, elec_df)

df1 Review Mean:  6.918853820598014
df2 Review Mean:  6.93458584750412
Difference:  -0.015732026906106533
T-Statistic: -0.36421091026425517
P-Value: 0.7157164040794012


In [23]:
bootstrap(pop_df, elec_df)

df1 Means Stdv:  0.12960823465615803
df2 Means Stdv:  0.13263724162155813
df1 2.5 Percentile:  6.727902298850575
df1 97.5 Percentile:  7.2058620689655175
df2 2.5 Percentile:  6.704022988505747
df2 97.5 Percentile:  7.171379310344827
